In [1]:
! git clone https://git.wur.nl/bioinformatics/grs34806-deep-learning-project-data.git
! git clone https://github.com/maussn/GRS34806-project.git

fatal: destination path 'grs34806-deep-learning-project-data' already exists and is not an empty directory.
Cloning into 'GRS34806-project'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 34 (delta 10), reused 24 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (34/34), 13.14 KiB | 2.63 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [17]:
import sys
import os
from pathlib import Path

module_path = Path('GRS34806-project') / 'maurits'
if str(module_path) not in sys.path:
    sys.path.append(str(module_path))

# Now you can import data_loading
import data_loading as dl

os.chdir(Path('grs34806-deep-learning-project-data'))

In [21]:
import torch.nn as nn

In [18]:
batch_size = 5
num_steps = 10

# Example for one of the simulated datasets
datalist, labellist = dl.read("len100_200_n1000.seq", "len100_200_n1000.pos")
traindatalist, trainlabellist, testdatalist, testlabellist = dl.generate_train_test(datalist, labellist)
traindataset = [traindatalist, trainlabellist]
testdataset = [testdatalist, testlabellist]

# Set batch_size and num_steps (maximum sequence length)
train_iter = dl.load_data(batch_size, num_steps, traindataset)
test_iter = dl.load_data(batch_size, num_steps, testdataset)

print(next(iter(train_iter)))

[tensor([[10, 17,  3,  2,  1,  5,  3, 16,  5,  7],
        [10,  3,  5,  5, 15, 17,  7, 15, 13, 17],
        [10, 13,  6, 17, 13, 19,  8, 11, 10, 15],
        [10, 12, 17,  2, 17, 17,  6, 15,  7, 15],
        [10,  6,  3, 12,  6,  9,  5, 19, 11,  9]]), tensor([False, False, False,  True, False])]


In [19]:
from transformers import BertModel, BertTokenizer

tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False)
base_model = BertModel.from_pretrained("Rostlab/prot_bert")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/361 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

In [20]:
base_model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30, 1024, padding_idx=0)
    (position_embeddings): Embedding(40000, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-29): 30 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.0, i

In [ ]:
def pad_or_trim(seq, size):
    if len(seq) > size:
        return seq[:size]
    else:
        return seq + '_' * (size - len(seq))


def add_spaces_between_characters(seq):
    return ''.join(f'{s} ' for s in seq)[:-1]


seq = pad_or_trim(seq, context_size)
seq = add_spaces_between_characters(seq)
seq = re.sub(r"[UZOB]", "X", seq) # Translate non-canonical AAs to X
tokens = tokenizer(seq, return_tensors='pt')

In [ ]:
class ProtBERTClassifier(nn.Module):
    def __init__(self, base_model, num_hiddens):
        for parameter in base_model.parameters():
            parameter.requires_grad = False
        self.base_model = base_model
        self.output = nn.Sequential(
            nn.Linear(num_hiddens, 2)
        )

    def forward(self, tokens):
        embeddings = self.base_model(tokens).last_hidden_state
        return self.output(embeddings)
